<a href="https://colab.research.google.com/github/ssuxmin/sumin/blob/main/hierarchy_preprocessing_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment

In [ ]:
# Essentials
import numpy as np
import pandas as pd
import os, warnings, random, pickle

# Tools
import sklearn
from sklearn.preprocessing import LabelEncoder
import torch

# Model
import lightgbm as lgb

# Misc
warnings.filterwarnings('ignore')

# Path
processed_data_dir = './processed/'
model_dir = './models/'
predict_dir = './lgbm_predict/'

# 개발 환경 및 라이브러리 버전

In [ ]:
import platform, sys
print('[개발환경]')
print(f"윈도우 버전: {platform.system()} {platform.version()}")
print(f"파이썬 버전: {sys.version}")
print(' ')
print('[라이브러리 버전]')
print(f'Pandas Version: {pd.__version__}')
print(f'Numpy Version: {np.__version__}')
print(f'LightGBM Version: {lgb.__version__}')
print(f'Scikit-learn Version: {sklearn.__version__}')

[개발환경]
윈도우 버전: Windows 10.0.22621
파이썬 버전: 3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
 
[라이브러리 버전]
Pandas Version: 1.5.2
Numpy Version: 1.23.5
LightGBM Version: 3.3.5
Scikit-learn Version: 1.2.1


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

# Preprocessing

## Base

In [ ]:
train_data = pd.read_csv('data/train.csv')
brand_meta = pd.read_csv('data/brand_keyword_cnt.csv')
sales_data = pd.read_csv('data/sales.csv')

In [ ]:
def reduce_mem_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif col == 'date':
            pass
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return

In [ ]:
reduce_mem_usage(train_data)

Memory usage of dataframe is 56.37 MB
Memory usage after optimization is: 15.70 MB
Decreased by 72.1%


In [ ]:
reduce_mem_usage(brand_meta)

Memory usage of dataframe is 11.13 MB
Memory usage after optimization is: 2.93 MB
Decreased by 73.6%


In [ ]:
reduce_mem_usage(sales_data)

Memory usage of dataframe is 56.37 MB
Memory usage after optimization is: 28.74 MB
Decreased by 49.0%


In [ ]:
train_data.columns

Index(['ID', '제품', '대분류', '중분류', '소분류', '브랜드', '2022-01-01', '2022-01-02',
       '2022-01-03', '2022-01-04',
       ...
       '2023-03-26', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30',
       '2023-03-31', '2023-04-01', '2023-04-02', '2023-04-03', '2023-04-04'],
      dtype='object', length=465)

In [ ]:
# Label Encoding
# train
label_encoder = LabelEncoder()
categorical_columns = ['제품','대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

# sales
label_encoder = LabelEncoder()

for col in categorical_columns:
    label_encoder.fit(sales_data[col])
    sales_data[col] = label_encoder.transform(sales_data[col])

# brand_keyword_cnt
label_encoder = LabelEncoder()

label_encoder.fit(brand_meta['브랜드'])
brand_meta['브랜드'] = label_encoder.transform(brand_meta['브랜드'])

brand_meta = brand_meta.fillna(0)
brand_meta.isna().sum().sum()

main_list = train_data['대분류'].unique()
mid_list = train_data['중분류'].unique()
sub_list = train_data['소분류'].unique()
brand_list = train_data['브랜드'].unique()

In [ ]:
# predict date 컬럼 생성
pred_p = pd.date_range('2023-04-05', periods=21)
pred_p = pred_p.astype('str')
train_data[pred_p] = np.nan
brand_meta[pred_p] = np.nan

In [ ]:
# price features
sales_mean = (sales_data.iloc[:,6:-21]/train_data.iloc[:,6:-21]).mean(axis=1)
sales_max = (sales_data.iloc[:,6:-21]/train_data.iloc[:,6:-21]).max(axis=1)
sales_min = (sales_data.iloc[:,6:-21]/train_data.iloc[:,6:-21]).min(axis=1)
sales_std = (sales_data.iloc[:,6:-21]/train_data.iloc[:,6:-21]).std(axis=1)


train_data['price mean'] = sales_mean
train_data['price std'] = sales_std
train_data['max price'] = sales_max
train_data['min price'] = sales_min

In [ ]:
# 구조 변경
td_melt = pd.melt(train_data, id_vars=['제품','ID','대분류', '중분류','소분류','브랜드','price mean','price std', 'max price','min price'])
td_melt.rename(columns={'variable':'date'}, inplace=True)
td_melt = td_melt.sort_values(by=['ID','브랜드','date'], ignore_index=True)

In [ ]:
td_melt['brand meta'] = np.nan
for i in brand_list:
    brand_len = len(td_melt.loc[td_melt['브랜드']==i, 'brand meta'])
    nums = brand_len / 480
    td_melt.loc[td_melt['브랜드']==i, 'brand meta'] = np.tile(brand_meta.loc[brand_meta['브랜드']==i].iloc[:,1:].values.reshape(-1), int(nums))

In [ ]:
# 여러 분류별 데이터 생성
time_idx = train_data.columns[6:-25]
main_data_avg = train_data.groupby('대분류')[time_idx].mean().reset_index()
mid_data_avg = train_data.groupby('중분류')[time_idx].mean().reset_index()
sub_data_avg = train_data.groupby('소분류')[time_idx].mean().reset_index()
brand_data_avg = train_data.groupby('브랜드')[time_idx].mean().reset_index()
main_mid_data_avg = train_data.groupby(['대분류','중분류'])[time_idx].mean().reset_index()
main_brand_data_avg = train_data.groupby(['대분류','브랜드'])[time_idx].mean().reset_index()
mid_brand_data_avg = train_data.groupby(['중분류','브랜드'])[time_idx].mean().reset_index()
sub_brand_data_avg = train_data.groupby(['소분류','브랜드'])[time_idx].mean().reset_index()

main_data_avg[pred_p] = np.nan
mid_data_avg[pred_p] = np.nan
sub_data_avg[pred_p] = np.nan
brand_data_avg[pred_p] = np.nan
main_mid_data_avg[pred_p] = np.nan
main_brand_data_avg[pred_p] = np.nan
mid_brand_data_avg[pred_p] = np.nan
sub_brand_data_avg[pred_p] = np.nan

In [ ]:
td_melt['main mean'] = np.nan

for i in main_list:
    brand_len = len(td_melt.loc[td_melt['대분류']==i, 'main mean'])
    nums = brand_len / 480
    td_melt.loc[td_melt['대분류']==i, 'main mean'] = np.tile(main_data_avg.loc[main_data_avg['대분류']==i].iloc[:,1:].values.reshape(-1), int(nums))

td_melt['mid mean'] = np.nan

for i in mid_list:
    brand_len = len(td_melt.loc[td_melt['중분류']==i, 'mid mean'])
    nums = brand_len / 480
    td_melt.loc[td_melt['중분류']==i, 'mid mean'] = np.tile(mid_data_avg.loc[mid_data_avg['중분류']==i].iloc[:,1:].values.reshape(-1), int(nums))

td_melt['sub mean'] = np.nan

for i in sub_list:
    brand_len = len(td_melt.loc[td_melt['소분류']==i, 'sub mean'])
    nums = brand_len / 480
    td_melt.loc[td_melt['소분류']==i, 'sub mean'] = np.tile(sub_data_avg.loc[sub_data_avg['소분류']==i].iloc[:,1:].values.reshape(-1), int(nums))

td_melt['brand mean'] = np.nan

for i in brand_list:
    brand_len = len(td_melt.loc[td_melt['브랜드']==i, 'brand mean'])
    nums = brand_len / 480
    td_melt.loc[td_melt['브랜드']==i, 'brand mean'] = np.tile(brand_data_avg.loc[brand_data_avg['브랜드']==i].iloc[:,1:].values.reshape(-1), int(nums))

td_melt['main mid mean'] = np.nan

for i in main_list:
    main_mid_list = td_melt[td_melt['대분류']==i]['중분류'].unique()
    for j in main_mid_list:
        brand_len = len(td_melt.loc[(td_melt['대분류']==i) & (td_melt['중분류']==j), 'main mid mean'])
        nums = brand_len / 480
        td_melt.loc[(td_melt['대분류']==i) & (td_melt['중분류']==j), 'main mid mean'] = np.tile(main_mid_data_avg.loc[(main_mid_data_avg['대분류']==i) & (main_mid_data_avg['중분류']==j)].iloc[:,2:].values.reshape(-1), int(nums))

td_melt['main brand mean'] = np.nan

for i in main_list:
    main_brand_list = td_melt[td_melt['대분류']==i]['브랜드'].unique()
    for j in main_brand_list:
        brand_len = len(td_melt.loc[(td_melt['대분류']==i) & (td_melt['브랜드']==j), 'main brand mean'])
        nums = brand_len / 480
        td_melt.loc[(td_melt['대분류']==i) & (td_melt['브랜드']==j), 'main brand mean'] = np.tile(main_brand_data_avg.loc[(main_brand_data_avg['대분류']==i) & (main_brand_data_avg['브랜드']==j)].iloc[:,2:].values.reshape(-1), int(nums))


td_melt['mid brand mean'] = np.nan

for i in mid_list:
    mid_brand_list = td_melt[td_melt['중분류']==i]['브랜드'].unique()
    for j in mid_brand_list:
        brand_len = len(td_melt.loc[(td_melt['중분류']==i) & (td_melt['브랜드']==j), 'mid brand mean'])
        nums = brand_len / 480
        td_melt.loc[(td_melt['중분류']==i) & (td_melt['브랜드']==j), 'mid brand mean'] = np.tile(mid_brand_data_avg.loc[(mid_brand_data_avg['중분류']==i) & (mid_brand_data_avg['브랜드']==j)].iloc[:,2:].values.reshape(-1), int(nums))

td_melt['sub brand mean'] = np.nan

for i in sub_list:
    sub_brand_list = td_melt[td_melt['소분류']==i]['브랜드'].unique()
    for j in sub_brand_list:
        brand_len = len(td_melt.loc[(td_melt['소분류']==i) & (td_melt['브랜드']==j), 'sub brand mean'])
        nums = brand_len / 480
        td_melt.loc[(td_melt['소분류']==i) & (td_melt['브랜드']==j), 'sub brand mean'] = np.tile(sub_brand_data_avg.loc[(sub_brand_data_avg['소분류']==i) & (sub_brand_data_avg['브랜드']==j)].iloc[:,2:].values.reshape(-1), int(nums))


In [ ]:
td_melt.to_pickle(processed_data_dir+'hierarchy_data_base.pkl')

## Lag & Rolling

In [ ]:
td_melt = pd.read_pickle(processed_data_dir+'hierarchy_data_base.pkl')

In [ ]:
reduce_mem_usage(td_melt)

Memory usage of dataframe is 1032.89 MB
Memory usage after optimization is: 400.06 MB
Decreased by 61.3%


In [ ]:
# 판매량 lag 1~21 컬럼 생성
value_lag_add = td_melt[['ID','date','value']]
lag_days = [i for i in range(1, 22)]

value_lag_add = value_lag_add.assign(**{
        '{}_lag_{}'.format(col, l): value_lag_add.groupby(['ID'])[col].transform(lambda x: x.shift(l))
        for l in lag_days
        for col in ['value']
    })

for col in list(value_lag_add):
    if 'lag' in col:
        value_lag_add[col] = value_lag_add[col].astype(np.float16)

hierarchy_data_lag_21 = td_melt.merge(value_lag_add)

In [ ]:
# 판매량 rolling features
hierarchy_data_lag_21_rolling_7_14_30 = hierarchy_data_lag_21.copy()

# value lag
hierarchy_data_lag_21_rolling_7_14_30['sales_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','value']].groupby(['ID'])['value'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sales_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','value']].groupby(['ID'])['value'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sales_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','value']].groupby(['ID'])['value'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sales_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','value']].groupby(['ID'])['value'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sales_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','value']].groupby(['ID'])['value'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sales_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','value']].groupby(['ID'])['value'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

# meta data
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand meta']].groupby(['ID'])['brand meta'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand meta']].groupby(['ID'])['brand meta'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand meta']].groupby(['ID'])['brand meta'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand meta']].groupby(['ID'])['brand meta'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand meta']].groupby(['ID'])['brand meta'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand meta']].groupby(['ID'])['brand meta'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

In [ ]:
reduce_mem_usage(hierarchy_data_lag_21_rolling_7_14_30)

Memory usage of dataframe is 1462.05 MB
Memory usage after optimization is: 1898.48 MB
Decreased by -29.9%


In [ ]:
# more rolling features
hierarchy_data_lag_21_rolling_7_14_30['main_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mean']].groupby(['ID'])['main mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mean']].groupby(['ID'])['main mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mean']].groupby(['ID'])['main mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mean']].groupby(['ID'])['main mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mean']].groupby(['ID'])['main mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mean']].groupby(['ID'])['main mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

hierarchy_data_lag_21_rolling_7_14_30['mid_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid mean']].groupby(['ID'])['mid mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid mean']].groupby(['ID'])['mid mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid mean']].groupby(['ID'])['mid mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid mean']].groupby(['ID'])['mid mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid mean']].groupby(['ID'])['mid mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid mean']].groupby(['ID'])['mid mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

hierarchy_data_lag_21_rolling_7_14_30['sub_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub mean']].groupby(['ID'])['sub mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub mean']].groupby(['ID'])['sub mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub mean']].groupby(['ID'])['sub mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub mean']].groupby(['ID'])['sub mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub mean']].groupby(['ID'])['sub mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub mean']].groupby(['ID'])['sub mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand mean']].groupby(['ID'])['brand mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand mean']].groupby(['ID'])['brand mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand mean']].groupby(['ID'])['brand mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand mean']].groupby(['ID'])['brand mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand mean']].groupby(['ID'])['brand mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['brand_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','brand mean']].groupby(['ID'])['brand mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

hierarchy_data_lag_21_rolling_7_14_30['main_mid_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mid mean']].groupby(['ID'])['main mid mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_mid_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mid mean']].groupby(['ID'])['main mid mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_mid_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mid mean']].groupby(['ID'])['main mid mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_mid_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mid mean']].groupby(['ID'])['main mid mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_mid_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mid mean']].groupby(['ID'])['main mid mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_mid_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main mid mean']].groupby(['ID'])['main mid mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

hierarchy_data_lag_21_rolling_7_14_30['main_brand_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main brand mean']].groupby(['ID'])['main brand mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_brand_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main brand mean']].groupby(['ID'])['main brand mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_brand_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main brand mean']].groupby(['ID'])['main brand mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_brand_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main brand mean']].groupby(['ID'])['main brand mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_brand_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main brand mean']].groupby(['ID'])['main brand mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['main_brand_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','main brand mean']].groupby(['ID'])['main brand mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

hierarchy_data_lag_21_rolling_7_14_30['mid_brand_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid brand mean']].groupby(['ID'])['mid brand mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_brand_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid brand mean']].groupby(['ID'])['mid brand mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_brand_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid brand mean']].groupby(['ID'])['mid brand mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_brand_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid brand mean']].groupby(['ID'])['mid brand mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_brand_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid brand mean']].groupby(['ID'])['mid brand mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['mid_brand_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','mid brand mean']].groupby(['ID'])['mid brand mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

hierarchy_data_lag_21_rolling_7_14_30['sub_brand_rolling_mean_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub brand mean']].groupby(['ID'])['sub brand mean'].transform(lambda x: x.shift(21).rolling(7).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_brand_rolling_mean_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub brand mean']].groupby(['ID'])['sub brand mean'].transform(lambda x: x.shift(21).rolling(14).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_brand_rolling_mean_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub brand mean']].groupby(['ID'])['sub brand mean'].transform(lambda x: x.shift(21).rolling(30).mean()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_brand_rolling_std_7']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub brand mean']].groupby(['ID'])['sub brand mean'].transform(lambda x: x.shift(21).rolling(7).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_brand_rolling_std_14']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub brand mean']].groupby(['ID'])['sub brand mean'].transform(lambda x: x.shift(21).rolling(14).std()).astype('float')
hierarchy_data_lag_21_rolling_7_14_30['sub_brand_rolling_std_30']=hierarchy_data_lag_21_rolling_7_14_30[['ID','date','sub brand mean']].groupby(['ID'])['sub brand mean'].transform(lambda x: x.shift(21).rolling(30).std()).astype('float')

In [ ]:
reduce_mem_usage(hierarchy_data_lag_21_rolling_7_14_30)

Memory usage of dataframe is 4604.36 MB
Memory usage after optimization is: 2509.48 MB
Decreased by 45.5%


In [ ]:
# 필요없는 컬럼 삭제
hierarchy_data_lag_21_rolling_7_14_30 = hierarchy_data_lag_21_rolling_7_14_30.drop(['brand meta', 'main mean',
       'mid mean', 'sub mean', 'brand mean', 'main mid mean',
       'main brand mean', 'mid brand mean', 'sub brand mean'], axis=1)

In [ ]:
hierarchy_data_lag_21_rolling_7_14_30.to_pickle(processed_data_dir+'hierarchy_lag_21_rolling_7_14_30.pkl')

# Prediction

In [ ]:
final_train = pd.read_pickle(processed_data_dir+'hierarchy_lag_21_rolling_7_14_30.pkl')

In [ ]:
# LightGBM parameters
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,  #learing_rate 0.01 ~ 0.1 일반적이지만 성능을 더 높이려면 더 줄여도 된다.
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1,
                }

In [ ]:
SEED = 42
seed_everything(SEED)
lgb_params['seed'] = SEED
predict_h = 21
remove_features = ['ID', 'date','value']
target = 'value'
features = [col for col in list(final_train) if col not in remove_features]
all_features = list(final_train)

In [ ]:
print('Train')
final_train_data = final_train.copy()

train_period = final_train_data['date'] <= '2023-04-04'
valid_period = (final_train_data['date'] <= '2023-02-27') & (final_train_data['date'] >= '2023-02-07')
pred_period = (final_train_data['date']>'2023-04-04') & (final_train_data['date'] <= '2023-04-25')

lgbm_train_data = lgb.Dataset(final_train_data[train_period][features], label=final_train_data[train_period][target])
lgbm_valid_data = lgb.Dataset(final_train_data[valid_period][features], label=final_train_data[valid_period][target])

pred_df = final_train_data[pred_period].reset_index(drop=True)
pred_df = pred_df[all_features]

pred_sales = pred_df[['date','value']]
substitute = pred_sales['value'].values
substitute[(pred_sales['date'] > '2023-04-04')] = np.nan
pred_df['value'] = substitute

pred_df.to_pickle(processed_data_dir + 'hierarchy_lag_21_nprod.pkl')
del pred_df, pred_sales, substitute

seed_everything(SEED)
estimator = lgb.train(lgb_params,
                        lgbm_train_data,
                        valid_sets = [lgbm_valid_data],
                        verbose_eval = 100,
                        early_stopping_rounds = 10
                        )
display(pd.DataFrame({'name':estimator.feature_name(),
                        'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))

model_name = model_dir+'hierarchy_lag_21_nprod1.bin'
pickle.dump(estimator, open(model_name, 'wb'))

del lgbm_train_data, lgbm_valid_data, estimator
gc.collect()

Train
Training until validation scores don't improve for 10 rounds
[100]	valid_0's rmse: 116.326
[200]	valid_0's rmse: 102.43
[300]	valid_0's rmse: 100.027
[400]	valid_0's rmse: 97.614
[500]	valid_0's rmse: 95.7059
[600]	valid_0's rmse: 93.4323
[700]	valid_0's rmse: 91.6534
[800]	valid_0's rmse: 89.1687
[900]	valid_0's rmse: 87.6472
[1000]	valid_0's rmse: 85.6172
[1100]	valid_0's rmse: 84.2846
Early stopping, best iteration is:
[1183]	valid_0's rmse: 83.2997


,name,imp
8,min_price,27140
9,value_lag_1,26294
7,max_price,26285
5,price_mean,24755
6,price_std,24535
44,main_rolling_mean_30,21814
46,main_rolling_std_14,21058
47,main_rolling_std_30,20853
42,main_rolling_mean_7,20134
45,main_rolling_std_7,19870


130

In [ ]:
base_test = pd.read_pickle(processed_data_dir+'hierarchy_lag_21_nprod.pkl')
value_col = [col for col in list(base_test) if 'e_lag' in col]
pred_p = pd.date_range('2023-04-05', periods=21)
pred_p = pred_p.astype('str')
main_time = time.time()
all_preds = pd.DataFrame()
col_len = len(value_col)

for j, PREDICT_DAY in enumerate(pred_p):
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()


    grid_df = base_test.copy()


    model_path = model_dir+'hierarchy_lag_21_nprod'+str(VER)+'.bin'

    estimator = pickle.load(open(model_path, 'rb'))

    mask = base_test['date']==PREDICT_DAY

    base_test[target][mask] = estimator.predict(grid_df[mask][features])

    pred_value = pred_p[j+1:j+22]
    col_len = col_len - 1
    for i in range(col_len):
        a = base_test[base_test['date']==pred_value[i]]
        a.loc[:,value_col[i]] = base_test[target][mask].values
        base_test[base_test['date']==pred_value[i]] = a

    temp_df = base_test[mask][['ID',target]]
    temp_df.columns = ['ID',PREDICT_DAY]
    if 'ID' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['ID'], how='left')
    else:
        all_preds = temp_df.copy()

    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df[PREDICT_DAY].sum()))

    del temp_df

all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 2023-04-05
##########  0.07 min round |  0.07 min total |  178447.62 day sales |
Predict | Day: 2023-04-06
##########  0.06 min round |  0.12 min total |  189538.87 day sales |
Predict | Day: 2023-04-07
##########  0.06 min round |  0.18 min total |  192281.15 day sales |
Predict | Day: 2023-04-08
##########  0.05 min round |  0.23 min total |  193858.99 day sales |
Predict | Day: 2023-04-09
##########  0.05 min round |  0.28 min total |  198547.77 day sales |
Predict | Day: 2023-04-10
##########  0.05 min round |  0.33 min total |  210512.18 day sales |
Predict | Day: 2023-04-11
##########  0.05 min round |  0.39 min total |  218212.57 day sales |
Predict | Day: 2023-04-12
##########  0.05 min round |  0.44 min total |  224590.17 day sales |
Predict | Day: 2023-04-13
##########  0.05 min round |  0.48 min total |  226623.91 day sales |
Predict | Day: 2023-04-14
##########  0.04 min round |  0.53 min total |  216680.61 day sales |
Predict | Day: 2023-04-15
##########  0.

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0.011685,0.211079,0.536388,0.681028,0.892817,0.987067,0.987189,1.032052,1.047019,...,1.144889,1.138560,1.136776,1.305445,1.159696,1.161276,1.174930,1.211935,1.205769,1.135613
1,1,1.572657,1.665551,2.117700,1.818782,1.785662,2.090107,2.092046,2.195306,2.123365,...,2.130509,2.142735,2.098483,2.267295,2.251230,2.025134,2.006642,2.003369,1.916594,1.849224
2,2,0.076007,0.709047,1.021442,0.949609,1.102696,1.231939,1.213793,1.257149,1.154665,...,1.117415,1.116513,1.096715,1.266667,1.177796,1.027374,1.026684,1.003284,0.988961,0.939907
3,3,0.079732,0.725790,1.109040,1.064037,1.201856,1.367784,1.320453,1.368186,1.243823,...,1.147279,1.148640,1.126987,1.311449,1.231136,1.057542,1.054485,1.012922,0.989969,0.984070
4,4,0.154983,0.970032,1.434157,1.139261,1.217798,1.172406,1.271298,1.139129,1.172528,...,1.299966,1.281827,1.274007,1.526929,2.356987,2.278691,1.768476,1.840737,1.969875,1.816731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,0.610858,0.781836,0.962917,1.074318,1.186556,1.320945,1.367964,1.370963,1.338355,...,1.338791,1.290671,1.283066,1.467678,1.456775,1.308082,1.248993,1.275552,1.234901,1.191348
15886,15886,6.047615,8.725229,11.175869,12.694461,14.015157,13.087048,12.126351,12.773513,12.287257,...,12.595052,12.219381,11.374771,10.817956,11.770424,12.723664,13.105445,12.000196,10.590330,9.873206
15887,15887,0.057155,0.626825,0.881882,0.911484,1.081310,1.292774,1.296142,1.405508,1.194115,...,1.261190,1.260928,1.239593,1.235750,1.236179,1.153339,1.087072,1.032171,1.045222,1.002195
15888,15888,1.019816,1.150706,0.981649,1.178955,1.343666,1.339387,1.341114,1.415440,1.226341,...,1.251242,1.292354,1.273932,1.214053,1.230640,1.102552,1.078811,1.014558,1.046475,0.957062


In [ ]:
submission = pd.read_csv('data/sample_submission.csv')

submit = submission[['ID']].merge(all_preds, on=['ID'], how='left').fillna(0)
submit = round(submit, 0).astype(int)
submit.to_csv(predict_dir+'./hierarchy_predict_nprod.csv', index=False)
submit

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,3,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,0,1,1,1,1,1,1,1,1,...,1,1,1,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15886,15886,6,9,11,13,14,13,12,13,12,...,13,12,11,11,12,13,13,12,11,10
15887,15887,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15888,15888,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
